In [1]:
from delphi_epidata import Epidata
import pandas as pd

In [2]:
df = pd.DataFrame()

In [3]:
# Data has to be acquired in batches since there is a length limit in the API

# 8/04/2020 - 15/04/2020
res = Epidata.covidcast('google-survey', 'smoothed_cli', 'day', 'county', [Epidata.range(20200408, 20200415)], geo_value='*')
print(res['result'], res['message'], len(res['epidata']))
df = df.append(res['epidata'])

1 success 3235


In [4]:
# 16/04/2020 - 19/04/2020
res = Epidata.covidcast('google-survey', 'smoothed_cli', 'day', 'county', [Epidata.range(20200416, 20200419)], geo_value='*')
print(res['result'], res['message'], len(res['epidata']))
df = df.append(res['epidata'])

1 success 2588


In [5]:
# 20/04/2020 - 24/04/2020
res = Epidata.covidcast('google-survey', 'smoothed_cli', 'day', 'county', [Epidata.range(20200420, 20200424)], geo_value='*')
print(res['result'], res['message'], len(res['epidata']))
df = df.append(res['epidata'])

1 success 3235


In [6]:
# 25/04/2020 - 29/04/2020
res = Epidata.covidcast('google-survey', 'smoothed_cli', 'day', 'county', [Epidata.range(20200425, 20200429)], geo_value='*')
print(res['result'], res['message'], len(res['epidata']))
df = df.append(res['epidata'])

1 success 3186


In [7]:
# 30/04/2020 - 03/05/2020
res = Epidata.covidcast('google-survey', 'smoothed_cli', 'day', 'county', [Epidata.range(20200430, 20200503)], geo_value='*')
print(res['result'], res['message'], len(res['epidata']))
df = df.append(res['epidata'])

1 success 2584


In [8]:
# 04/05/2020 - 07/05/2020
res = Epidata.covidcast('google-survey', 'smoothed_cli', 'day', 'county', [Epidata.range(20200504, 20200507)], geo_value='*')
print(res['result'], res['message'], len(res['epidata']))
df = df.append(res['epidata'])

1 success 2588


In [9]:
# 08/05/2020 - 12/05/2020
res = Epidata.covidcast('google-survey', 'smoothed_cli', 'day', 'county', [Epidata.range(20200508, 20200512)], geo_value='*')
print(res['result'], res['message'], len(res['epidata']))
df = df.append(res['epidata'])

1 success 3235


In [10]:
# 13/05/2020 - 17/05/2020
res = Epidata.covidcast('google-survey', 'smoothed_cli', 'day', 'county', [Epidata.range(20200513, 20200517)], geo_value='*')
print(res['result'], res['message'], len(res['epidata']))
df = df.append(res['epidata'])

1 success 1005


In [11]:
counties = []
for county in df['geo_value']:
    counties.append(county)
counties = list(set(counties))
cleanedDf = pd.DataFrame(dtype=str)

In [12]:
for county in counties:
    countyData = df[df['geo_value'] == county]
    countyData = countyData.reset_index()
    countyData = countyData.drop(['index'], axis=1)
    
    # Time series is in rows unlike applemobility data, converting to columns instead
    temp = pd.DataFrame(columns=['geo_value', *countyData['time_value']])
    
    # Renaming columns to match applemobility data
    columns = ["geo_value"]
    for i,column in enumerate(temp.columns):
        if i == 0:
            continue
        column = list(str(column))
        column.insert(4, '-')
        column.insert(7, '-')
        columns.append(''.join(column))
    temp.columns = columns
    
    values = []
    for i,column in enumerate(columns):
        if i == 0:
            values.append(str(countyData['geo_value'].values[0]))
            continue
        else:
            columnAdj = int(column.replace('-', ''))
            value = countyData[countyData['time_value'] == columnAdj].values[0][3]
            values.append(value)
    
    temp = temp.append(pd.Series(values, index=temp.columns), ignore_index=True)
    cleanedDf = cleanedDf.append(temp, ignore_index=True)

In [13]:
cleanedDf = cleanedDf.reindex(sorted(cleanedDf.columns), axis=1)
cleanedDf = cleanedDf.fillna(0)
firstCol = cleanedDf.pop('geo_value')
cleanedDf.insert(0, 'geo_value', firstCol)

In [14]:
cleanedDf.to_csv('covidsymptoms_google.csv')